<a href="https://colab.research.google.com/github/amolaka/DS-3001---Voting-Project/blob/main/va_voting_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Voting in Virginia Starter Notebook

This is just a starter notebook with some "heatmap"/choropleth plots of Virginia and the 2020 election.

In [ ]:
! pip3 install mapclassify

In [ ]:
pip install --upgrade pip


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.5 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2


In [ ]:
pip install --upgrade geopandas fiona gdal


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.6/802.6 kB 5.5 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
import geopandas as gpd
import mapclassify
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable

# Load data
gdf = gpd.read_file('./VirginiaCounty_ClippedToShoreline.shx') # Load Virginia shapefile for maps
vdf = pd.read_csv('./voting_VA.csv') # Load historical voting data
cdf = pd.read_csv('./county_adjacencies.csv') # Load county adjacency data and population data

# Merge shapefile data and county adjacency data:
gdf['FIPS_left'] = pd.to_numeric(gdf['STCOFIPS'])
df = gdf.merge(cdf,left_on='FIPS_left',right_on='FIPS')  # merge county population and adjacency data into df
del gdf, cdf

ERROR:fiona._env:./VirginiaCounty_ClippedToShoreline.shx: No such file or directory


DriverError: ./VirginiaCounty_ClippedToShoreline.shx: No such file or directory

# Population Data

Here is code to make choropleth maps:

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0, 0, 2, 2])
divider = make_axes_locatable(ax)
ax.axis('off')
cax = divider.append_axes("right", size="2%", pad=0.08)
df.plot(ax=ax,column='Population2022',legend=True,cax=cax)

# Voting Districts

My map:

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0, 0, 2, 2])
divider = make_axes_locatable(ax)
ax.axis('off')
#cax = divider.append_axes("right", size="2%", pad=0.08)
df['Districts'] = df['District'].astype(str)
df.plot(ax=ax,column='Districts',legend=True)#,cax=cax

Actual map:

<img src="https://github.com/phillipsophie/project_voting/blob/main/va_map.jpeg?raw=1" width="1350" >

This is close but there are some issues to work out.

## Voting from 2020

Clean vote data and merge into the df we're using to make maps:

In [ ]:
df20 = vdf.loc[ vdf['year']==2020,:] # Only the 2020 data

Dvotes = df20.loc[(df20['party']=='DEMOCRAT'),:].groupby('county_fips')['candidatevotes'].sum()
Dvotes = Dvotes.rename('dem_votes_2020')
df = df.merge(Dvotes,left_on='FIPS_left',right_on='county_fips') #df1.merge(df2, left_on='lkey', right_on='rkey

Rvotes = df20.loc[(df20['party']=='REPUBLICAN'),:].groupby('county_fips')['candidatevotes'].sum()
Rvotes = Rvotes.rename('rep_votes_2020')
df = df.merge(Rvotes,left_on='FIPS_left',right_on='county_fips') #df1.merge(df2, left_on='lkey', right_on='rkey

Raw 2020 Democrat votes:

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0, 0, 2, 2])
divider = make_axes_locatable(ax)
ax.axis('off')
cax = divider.append_axes("right", size="2%", pad=0.08)
df.plot(ax=ax,column='dem_votes_2020',legend=True,cax=cax)

Raw Republican votes:

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0, 0, 2, 2])
divider = make_axes_locatable(ax)
ax.axis('off')
cax = divider.append_axes("right", size="2%", pad=0.08)
df.plot(ax=ax,column='rep_votes_2020',legend=True,cax=cax)

Since both more D's and R's live places where more people live, you get really similar maps. But if you look at the scales, they're very different. We need to represent not just the raw levels, but the vote margins between the two parties.

Difference in Dem and Rep votes:

In [ ]:
df['vote_diff'] = df['rep_votes_2020']-df['dem_votes_2020']

fig = plt.figure()
ax = fig.add_axes([0, 0, 2, 2])
divider = make_axes_locatable(ax)
ax.axis('off')
cax = divider.append_axes("right", size="2%", pad=0.08)
df.plot(ax=ax,column='vote_diff',legend=True,cax=cax,cmap='bwr',edgecolor='k')

That plot is somewhat misleading because the "white" color values don't correspond to zero, but rather -100,000, making the state look more red than it is. Because of the psychology of the visualization, it's probably better not to pick red/blue colors:

In [ ]:
df['vote_diff'] = df['rep_votes_2020']-df['dem_votes_2020']

fig = plt.figure()
ax = fig.add_axes([0, 0, 2, 2])
divider = make_axes_locatable(ax)
ax.axis('off')
cax = divider.append_axes("right", size="2%", pad=0.08)
df.plot(ax=ax,column='vote_diff',legend=True,cax=cax,edgecolor='w')

Here's a way to discretize the scale:

In [ ]:
df['vote_diff'] = df['rep_votes_2020']-df['dem_votes_2020']

fig = plt.figure()
ax = fig.add_axes([0, 0, 2, 2])
divider = make_axes_locatable(ax)
ax.axis('off')
#cax = divider.append_axes("right", size="2%", pad=0.08)
df.plot(ax=ax,column='vote_diff',legend=True,cmap='bwr',edgecolor='k',scheme='Quantiles',k=7)

Slightly better, still not perfect.

That is a very long tail. Support for the Democrats is extremely strong in the north-northeast and southeast parts of the state.

Let's take the inverse hyperbolic sine of the difference to resolve some of the "long tail" issues:

In [ ]:
df['vote_diff_ihs'] = np.arcsinh(df['vote_diff'])

fig = plt.figure()
ax = fig.add_axes([0, 0, 2, 2])
divider = make_axes_locatable(ax)
ax.axis('off')
cax = divider.append_axes("right", size="2%", pad=0.08)
df.plot(ax=ax,column='vote_diff_ihs',legend=True,cax=cax,cmap='bwr',edgecolor='k')

This is pretty good. This gives a little more information about where Democrat strongholds are, versus Republican strongholds, and the intensities are in "log votes" rather than percentages, so the picture is easier to interpret.

We could also compute the percent difference:
$$
\dfrac{Dem - Rep}{Rep}
$$


In [ ]:
df['pct_diff'] = -df['vote_diff']/df['rep_votes_2020']

fig = plt.figure()
ax = fig.add_axes([0, 0, 2, 2])
divider = make_axes_locatable(ax)
ax.axis('off')
cax = divider.append_axes("right", size="2%", pad=0.08)
df.plot(ax=ax,column='pct_diff',legend=True,cax=cax)

Can you see Charlottesville, Richmond, Fairfax, and Petersburg? The danger in looking at percent difference, though, is that a small place might overwhelmingly prefer one party, but they don't have enough votes to carry the election if they are merged with a nearby political unit that is much larger and more purple. That's why looking at the arcsinh of vote differences gives a pretty useful guide not just to the intensity of support in terms of local opinion, but the vote margin as well.

Notice that if we compute the percent difference in the opposite direction, the pictures seems to tell a slightly different story?

In [ ]:
df['pct_diff'] = df['vote_diff']/df['dem_votes_2020']

fig = plt.figure()
ax = fig.add_axes([0, 0, 2, 2])
divider = make_axes_locatable(ax)
ax.axis('off')
cax = divider.append_axes("right", size="2%", pad=0.08)
df.plot(ax=ax,column='pct_diff',legend=True,cax=cax)

So these maps are fun, but you have to be careful to understand what you're looking at before jumping to conclusions.